In [1]:
# DONT USE THIS NOTEBOOK FOR WIKIKG90Mv2
# USE NOTEBOOK IN ogb_stuff FOLDER

import pickle

In [2]:
import os
from tqdm import tqdm
from typing import Dict
from collections import defaultdict
import numpy as np

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

def numLines(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1
def loadData(filename, max_points):
    file_len = numLines(filename)
    f = open(filename, 'r')
    inputs = []
    outputs = []
    for i in tqdm(range(file_len)):
        if i == max_points:
            break
        line = f.readline()
        if line[-1] == '\n':
            line = line[:-1]
        line = line.split('\t')
        inputs.append(line[0])
        outputs.append(line[1])
    data = {'inputs': inputs, 'outputs': outputs}
    return data
        
def load_entity_strings(filename):
    with open(filename) as f:
        lines = f.read().splitlines()
    return lines

def get_entity_wd_id_dict(filename):
    out = {}
    f = open(filename, 'r')
    for line in f:
        if line[-1] == '\n':
            line = line[:-1]
        line = line.split('\t')
        out[line[1]] = line[0]
    return out
    

def create_filter_dict(data) -> Dict[str, int]:
    filter_dict = defaultdict(list)
    for input, output in zip(data["inputs"], data["outputs"]):
        filter_dict[input].append(output)
    return filter_dict

def getAllFilteringEntities(input, filter_dicts):
    entities = []
    splits = ['train', 'test', 'valid']
    for s in splits:
        entities.extend(filter_dicts[s][input])
    return list(set(entities))

def wikidata_link_from_id(id):
    uri = 'https://www.wikidata.org/wiki/' + id
    return uri

In [3]:
dataset_name = 'wikikg90mv2'
# entity_strings = load_entity_strings(os.path.join("data", dataset_name, "entity_strings.txt"))

In [4]:
# entity_strings_set = set(entity_strings)
# len(entity_strings_set)

In [5]:
data = {}
# splits = ['train', 'valid', 'test']
splits = ['valid', 'test']
dataset_name = 'wnrr'
for split in splits:
    data[split] = loadData(os.path.join('data', dataset_name, split + '.txt'), -1)

100%|███████████████████████████████████████████████████████████████| 6268/6268 [00:00<00:00, 836139.48it/s]


In [6]:
filter_dicts = {}
# splits = ['train', 'valid', 'test']
splits = ['valid', 'test']
for split in splits:
    filter_dicts[split] = create_filter_dict(data[split])

In [40]:
# fname = 'scores.pickle'
# fname = 'scores/scores_full_codexm_small.pickle'
fname = 'scores/scores_wikikg90mv2_train_10k.pickle'
# fname = 'scores_500_base_trie.pickle'
scores_data = pickle.load(open(fname, 'rb'))

In [41]:
scores_data['prediction_strings'][0]

['Genotyping of Mycobacterium tuberculosis complex by variable-number tandem-repeat analysis of single-',
 'Whole-genome sequencing of Mycobacterium tuberculosis from the North-West of Iran.',
 'Rapid identification of rifampin-resistant Mycobacterium tuberculosis by rpoB gene mole',
 'The phylogeny of Mycobacterium tuberculosis (Ace): an assessment of species distribution.',
 'A study of Mycobacterium tuberculosis strains isolated in Tunisia.',
 'Characterization of Mycobacterium tuberculosis strains growing on TB treatment in Turkey.',
 'Evaluation of simple PCR methods for the genetic analysis of mycobacteria.',
 'Molecular characterization of Mycobacterium tuberculosis isolates from Pakistan using a polymorphic fingerprinting method.',
 'Molecular analysis of the Mycobacterium tuberculosis complex strains from human patients with pulmonary tuberculos',
 'Molecular epidemiology of tuberculosis in Germany, 1981-1971.',
 'Analysis of Mycobacterium tuberculosis strains in HIV, tubercul

In [42]:
predictions_scores_dicts = []
for string_arr, score_arr in tqdm(zip(scores_data['prediction_strings'], scores_data['scores'])):
    ps_pairs = [(p,s) for p,s in zip(string_arr, score_arr)]
    ps_pairs = list(set(ps_pairs)) # while sampling, duplicates are created
    # remove predictions that are not entities
    ps_dict_only_entities = defaultdict(list)
    for ps in ps_pairs:
#         if ps[0] in entity_strings_set:
        ps_dict_only_entities[ps[0]] = ps[1]
    predictions_scores_dicts.append(ps_dict_only_entities)

10000it [00:01, 7857.71it/s]


In [43]:
max([len(predictions_scores_dicts[x]) for x in range(500)])

300

In [44]:
id = 5007
print(scores_data['input_strings'][id], '|||', scores_data['target_strings'][id])
if scores_data['target_strings'][id] in scores_data['prediction_strings'][id]:
    print('Yes')
else:
    print('No')
print(set(scores_data['prediction_strings'][id]))
# print(scores_data['scores'][2])
# predictions_scores_dicts[id]

Manhattan tales: a dim echo of the 14th century.| published in ||| Medicine and Health, Rhode Island
No
{'Texas Heart Institute Journal', 'Journal of Cardiothoracic and Vascular Anesthesia', "Harvard women's health watch", 'Journal of the American College of Cardiology', 'Clinical Pediatrics', 'The Ohio State medical journal', 'Journal of the New Zealand Medical Association', 'Journal of the Canadian Dental Association', 'Journal of the Arkansas Medical Society', 'American Journal of Bioethics', 'Journal of clinical and experimental cardiac biomedicine : cardiology', 'The Illinois medical journal', 'Journal of the American College of Surgeons', 'Nursing mirror', 'Journal of Medical Ethics', 'Medical Clinics of North America', 'Nursing Outlook', 'Anaesthesia, critical care & pain medicine', 'Nursing mirror and midwives journal', 'Clinical and Investigative Medicine', 'Journal of Experimental Medicine', 'Journal of Infection in Developing Countries', 'Journal of Biological Chemistry', 'J

In [45]:
count = 0
for id in range(len(scores_data['input_strings'])):
    if scores_data['target_strings'][id] in scores_data['prediction_strings'][id]:
        count += 1
count

4502

In [46]:
# this works only if 1 prediction
for id in range(len(scores_data['input_strings'])):
    if scores_data['prediction_strings'][id] == scores_data['target_strings'][id]:
        print(scores_data['input_strings'][id], '|||', scores_data['target_strings'][id])

In [47]:
len(predictions_scores_dicts)

10000

In [48]:
import numpy as np
predictions_filtered = []
head_num_filter = 0
tail_num_filter = 0
hits_at_all = 0
count = 0
for i in tqdm(range(len(predictions_scores_dicts))):
    ps_dict = predictions_scores_dicts[i].copy()
    target = scores_data['target_strings'][i]
    inputs = scores_data['input_strings'][i]
    prediction_strings = ps_dict.keys()
    if target in prediction_strings:
        original_score = ps_dict[target]
    # get filtering entities
    filtering_entities = set()
    if len(filtering_entities) == 1:
        count += 1
    for ent in filtering_entities:
        if ent in ps_dict:
            ps_dict[ent] = -float("inf")
    if target in prediction_strings:
        ps_dict[target] = original_score
        hits_at_all += 1
    # softmax for scores
    names_arr = []
    scores_arr = []
    for k, v in ps_dict.items():
        names_arr.append(k)
        scores_arr.append(v)
    scores_arr = np.array(scores_arr)
#     scores_arr = softmax(scores_arr)
    for name, score in zip(names_arr, scores_arr):
        ps_dict[name] = score
    predictions_filtered.append(ps_dict)
print(head_num_filter/len(predictions_filtered), tail_num_filter/len(predictions_filtered))
print(hits_at_all/len(predictions_filtered))
print(count)

100%|███████████████████████████████████████████████████████████████| 10000/10000 [00:01<00:00, 9580.61it/s]

0.0 0.0
0.4502
0


In [49]:
count = {}
reciprocal_ranks = 0.0
k_list = [1,3,10]
for k in k_list:
    count[k] = 0
num_small_arrs = 0
count2 = 0
max_items_to_consider = 10
for i in range(len(predictions_filtered)):
    target = scores_data['target_strings'][i]
    ps_dict = predictions_filtered[i]
    ps_sorted = sorted(ps_dict.items(), key=lambda item: -item[1])
    ps_sorted = ps_sorted[:max_items_to_consider]
    if len(ps_dict) == 0:
        preds = []
    else:
        preds = [x[0] for x in ps_sorted]
    if target in preds:
        rank = preds.index(target) + 1
        reciprocal_ranks += 1./rank
    for k in k_list:
        if target in preds[:k]:
            count[k] += 1
    if len(preds) < 3 and target not in preds:
        num_small_arrs += 1
    if target in preds:
        count2 += 1
for k in k_list:
    hits_at_k = count[k]/len(predictions_filtered)
    print('hits@{}'.format(k), hits_at_k)
print('mrr', reciprocal_ranks/len(predictions_filtered))
print(num_small_arrs/len(predictions_filtered), 'were <3 length preds array without answer')

hits@1 0.3343
hits@3 0.386
hits@10 0.4187
mrr 0.3639429365079368
0.0013 were <3 length preds array without answer


In [121]:
count2/len(predictions_filtered)

0.30873333333333336

In [79]:
id = 0
inputs = scores_data['input_strings'][id]
preds = predictions_filtered[id]
preds = sorted(preds.items(), key=lambda item: -item[1])
target = scores_data['target_strings'][id]

In [106]:
print(inputs, 'Target:', target)
preds[:10], target

predict tail: trade name, NN -- a name given to a product or service | member of domain usage Target: metharbital, NN -- anticonvulsant drug (trade name Gemonil) used in the treatment of epilepsy


([('tolbutamide, NN -- sulfonylurea', -inf),
  ('serzone, NN -- an antidepressant drug (trade name Serzone)', -inf),
  ('procardia, NN -- calcium blocker (trade name Procardia)', -inf),
  ('sanitary towel, NN -- a disposable absorbent pad (trade name Kotex)',
   -inf),
  ('prosom, NN -- a frequently prescribed sleeping pill (trade name ProSom)',
   -inf)],
 'metharbital, NN -- anticonvulsant drug (trade name Gemonil) used in the treatment of epilepsy')

In [482]:
# only head/tails
count = 0
for id in range(60,120, 2):
    inputs = scores_data['input_strings'][id]
    preds = predictions_filtered[id]
    preds = sorted(preds.items(), key=lambda item: -item[1])
    target = scores_data['target_strings'][id]
    pred1 = preds[0][0]
    if pred1 == target:
        print(int(id/2), inputs, pred1)
        count += 1
'count', count

32 predict tail: ali kazemaini | birthplace | tehran
34 predict tail: ashta, maharashtra | instance of | human settlement
37 predict tail: roy shaw 0 | instance of | human being
40 predict tail: t. canby jones | has surname | jones (family name)
45 predict tail: naveen kumar | instance of | human being
46 predict tail: barlow respiratory hospital | host country | united states of america
48 predict tail: camiling | office held by head of government | mayor
51 predict tail: hazel soan | instance of | human being
52 predict tail: efrain herrera | sport played | association football
53 predict tail: oluf munck | instance of | human being
54 predict tail: thomas gilchrist | instance of | human being
58 predict tail: desmoplastic fibroma | subclass of | fibroma


('count', 12)

In [419]:
%%html
print("<a href='your_url_here'>Showing Text</a>")


In [364]:
e2wdid['pakistan']

'Q4121082'

In [174]:
sequences = ['english', 'english language', 'french']
t = Trie(sequences)

In [178]:
t.get('x')

[]